<a href="https://colab.research.google.com/github/Manpreet121990/manpreetcode/blob/main/lab%202%20mahd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np

# Load CIFAR-10 (preprocessing omitted for brevity, assume X_train, y_train, X_test, y_test are ready)
# Remember to resize images to 224x224 and one-hot encode labels
(X_train_cifar, y_train_cifar), (X_test_cifar, y_test_cifar) = tf.keras.datasets.cifar10.load_data()



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16 # or VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Needed for scenario 3 for augmentation
import numpy as np

# --- GPU Check (CRITICAL FOR SPEED) ---
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.is_built_with_cuda():
    print("TensorFlow is built with CUDA and should use GPU if available.")
else:
    print("TensorFlow is NOT built with CUDA. Training will be slow on CPU. Please enable GPU runtime.")
# --- End GPU Check ---

# 1. Load CIFAR-10 Dataset
(X_train_cifar, y_train_cifar), (X_test_cifar, y_test_cifar) = tf.keras.datasets.cifar10.load_data()

# Ensure X_train_cifar and X_test_cifar are float32 for consistency and normalization
X_train_cifar = X_train_cifar.astype(np.float32)
X_test_cifar = X_test_cifar.astype(np.float32)

# One-hot encode labels
num_classes = 10
y_train_one_hot = tf.keras.utils.to_categorical(y_train_cifar, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test_cifar, num_classes)

# --- Define Image Preprocessing and Batching ---
# IMPORTANT CHANGE HERE: Experiment with these sizes!
# Smaller values like (96, 96) or (64, 64) will be significantly faster.
TARGET_IMAGE_SIZE = (96, 96) # <<<-- CHANGE THIS TO (96, 96) or (64, 64) for speed up
BATCH_SIZE = 32 # Define your batch size

def preprocess_image(image, label):
    # Resize image
    image = tf.image.resize(image, TARGET_IMAGE_SIZE)
    # Normalize pixel values to [0, 1]
    image = image / 255.0
    return image, label

# Create train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_cifar, y_train_one_hot))
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000) # Shuffle the dataset
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE) # Prefetch for performance

# Create test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_cifar, y_test_one_hot))
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print(f"\n--- Data preparation complete. Images will be resized to {TARGET_IMAGE_SIZE}. ---")
print(f"Train Dataset batches: {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Test Dataset batches: {tf.data.experimental.cardinality(test_dataset).numpy()}")


# --- Scenario 1: Train model based on not updating the weights of pretrain model (Freeze all layers) ---
print("\n--- Starting Scenario 1: Training with Frozen VGG layers ---")

# IMPORTANT CHANGE HERE: Match input_shape to TARGET_IMAGE_SIZE
base_model_frozen = VGG16(weights='imagenet', include_top=False, input_shape=(TARGET_IMAGE_SIZE[0], TARGET_IMAGE_SIZE[1], 3))

# Freeze the convolutional layers
for layer in base_model_frozen.layers:
    layer.trainable = False

# Add new classification layers on top
x_frozen = base_model_frozen.output
x_frozen = Flatten()(x_frozen)
x_frozen = Dense(256, activation='relu')(x_frozen)
predictions_frozen = Dense(num_classes, activation='softmax')(x_frozen)

model_frozen = Model(inputs=base_model_frozen.input, outputs=predictions_frozen)

# Compile the model
model_frozen.compile(optimizer=Adam(learning_rate=0.001),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Train the model using the tf.data.Dataset
history_frozen = model_frozen.fit(train_dataset,
                                  epochs=5, # You can start with fewer epochs (e.g., 5) for quick testing
                                  validation_data=test_dataset)

print("\n--- Scenario 1: Training with frozen VGG layers complete ---")
print(f"Scenario 1 - Final Validation Accuracy: {history_frozen.history['val_accuracy'][-1]:.4f}")


# --- Scenario 2: Train based on updating partial layers of pretrain model (Partial Fine-tuning) ---
print("\n--- Starting Scenario 2: Training with Partially Unfrozen VGG layers (Fine-tuning) ---")

# Re-load base_model to ensure all layers are initially trainable, then selectively freeze
# IMPORTANT CHANGE HERE: Match input_shape to TARGET_IMAGE_SIZE
base_model_partial = VGG16(weights='imagenet', include_top=False, input_shape=(TARGET_IMAGE_SIZE[0], TARGET_IMAGE_SIZE[1], 3))

# Unfreeze layers from 'block5_conv1' onwards.
set_trainable = False
for layer in base_model_partial.layers:
    if layer.name == 'block5_conv1': # Unfreeze from this layer onwards
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Add new classification layers on top (same as Scenario 1)
x_partial = base_model_partial.output
x_partial = Flatten()(x_partial)
x_partial = Dense(256, activation='relu')(x_partial)
predictions_partial = Dense(num_classes, activation='softmax')(x_partial)

model_partial = Model(inputs=base_model_partial.input, outputs=predictions_partial)

# It's recommended to use a smaller learning rate for fine-tuning
model_partial.compile(optimizer=Adam(learning_rate=0.00001), # Smaller learning rate for fine-tuning
                      loss='categorical_crossentropy',

                      metrics=['accuracy'])

# Train the model using the tf.data.Dataset
history_partial = model_partial.fit(train_dataset,
                                    epochs=8, # Often requires more epochs than frozen layers
                                    validation_data=test_dataset)

print("\n--- Scenario 2: Training with partially unfrozen VGG layers complete ---")
print(f"Scenario 2 - Final Validation Accuracy: {history_partial.history['val_accuracy'][-1]:.4f}")



Num GPUs Available:  1
TensorFlow is built with CUDA and should use GPU if available.

--- Data preparation complete. Images will be resized to (96, 96). ---
Train Dataset batches: 1563
Test Dataset batches: 313

--- Starting Scenario 1: Training with Frozen VGG layers ---
Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 89s 52ms/step - accuracy: 0.6161 - loss: 1.1157 - val_accuracy: 0.7355 - val_loss: 0.7657
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 89s 56ms/step - accuracy: 0.7474 - loss: 0.7221 - val_accuracy: 0.7504 - val_loss: 0.7132
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 90s 57ms/step - accuracy: 0.7796 - loss: 0.6330 - val_accuracy: 0.7449 - val_loss: 0.7269
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 141s 56ms/step - accuracy: 0.8000 - loss: 0.5642 - val_accuracy: 0.7330 - val_loss: 0.7997
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 89s 56ms/step - accuracy: 0.8215 - loss: 0.5090 - val_accuracy: 0.7668 - val_loss: 0.7186

--- Scenario 1: Training with frozen VGG layers complete ---
Scenario

TypeError: ImageDataGenerator.flow() got an unexpected keyword argument 'target_size'

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# No longer strictly need ImageDataGenerator if we use tf.data.Dataset for augmentation
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt # For optional visualization

# --- GPU Check (CRITICAL FOR SPEED) ---
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.is_built_with_cuda():
    print("TensorFlow is built with CUDA and should use GPU if available.")
else:
    print("TensorFlow is NOT built with CUDA. Training will be slow on CPU. Please enable GPU runtime.")
# --- End GPU Check ---

# 1. Load CIFAR-10 Dataset
(X_train_cifar, y_train_cifar), (X_test_cifar, y_test_cifar) = tf.keras.datasets.cifar10.load_data()

# Ensure data types are float32 for image processing
X_train_cifar = X_train_cifar.astype(np.float32)
X_test_cifar = X_test_cifar.astype(np.float32)

# One-hot encode labels
num_classes = 10
y_train_one_hot = tf.keras.utils.to_categorical(y_train_cifar, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test_cifar, num_classes)

# --- Define Global Parameters ---
TARGET_IMAGE_SIZE = (96, 96) # Experiment with (224, 224), (96, 96), or (64, 64)
BATCH_SIZE = 32

# --- Preprocessing and Augmentation Functions for tf.data.Dataset ---

def augment_image(image, label):
    # Apply augmentations (TensorFlow ops are efficient)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    # Add more augmentations as needed
    # tf.image.random_rotation is not directly available, can be done with tf.keras.layers.RandomRotation
    # or custom logic using tf.raw_ops.ImageProjectiveTransformV2

    # Clip values to ensure they stay in [0, 1] after augmentations
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

def preprocess_and_augment(image, label):
    # Resize first
    image = tf.image.resize(image, TARGET_IMAGE_SIZE)
    # Normalize
    image = image / 255.0

    # Apply augmentation ONLY to training data
    # (The `map` function will be called on `train_dataset` only for augmentation)
    return augment_image(image, label)

def preprocess_only(image, label):
    # Resize first
    image = tf.image.resize(image, TARGET_IMAGE_SIZE)
    # Normalize
    image = image / 255.0
    return image, label

# --- Create tf.data.Dataset for Training (with augmentation) ---
train_dataset_augmented = tf.data.Dataset.from_tensor_slices((X_train_cifar, y_train_one_hot))
train_dataset_augmented = train_dataset_augmented.map(preprocess_and_augment, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset_augmented = train_dataset_augmented.shuffle(buffer_size=10000)
train_dataset_augmented = train_dataset_augmented.batch(BATCH_SIZE)
train_dataset_augmented = train_dataset_augmented.prefetch(tf.data.AUTOTUNE)

# --- Create tf.data.Dataset for Testing (no augmentation) ---
test_dataset_preprocessed = tf.data.Dataset.from_tensor_slices((X_test_cifar, y_test_one_hot))
test_dataset_preprocessed = test_dataset_preprocessed.map(preprocess_only, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset_preprocessed = test_dataset_preprocessed.batch(BATCH_SIZE)
test_dataset_preprocessed = test_dataset_preprocessed.prefetch(tf.data.AUTOTUNE)

print(f"\n--- Data preparation complete. Images will be resized to {TARGET_IMAGE_SIZE}. ---")
print(f"Train Dataset (augmented) batches: {tf.data.experimental.cardinality(train_dataset_augmented).numpy()}")
print(f"Test Dataset batches: {tf.data.experimental.cardinality(test_dataset_preprocessed).numpy()}")

# --- Scenario 3: Apply some layers on the top of model + use data augmentation and train the model ---
print("\n--- Starting Scenario 3: Fine-tuning with Data Augmentation (tf.data.Dataset) ---")

# Load base model
base_model_aug = VGG16(weights='imagenet', include_top=False, input_shape=(TARGET_IMAGE_SIZE[0], TARGET_IMAGE_SIZE[1], 3))

# Fine-tuning strategy: Unfreeze layers. Example: unfreeze from block5_conv1 onwards.
set_trainable_aug = False
for layer in base_model_aug.layers:
    if layer.name == 'block5_conv1': # Unfreeze from this layer onwards
        set_trainable_aug = True
    if set_trainable_aug:
        layer.trainable = True
    else:
        layer.trainable = False

# Add new classification layers on top
x_aug = base_model_aug.output
x_aug = Flatten()(x_aug)
x_aug = Dense(256, activation='relu')(x_aug)
predictions_aug = Dense(num_classes, activation='softmax')(x_aug)

model_aug = Model(inputs=base_model_aug.input, outputs=predictions_aug)

# Compile with a very small learning rate for fine-tuning
model_aug.compile(optimizer=Adam(learning_rate=0.000005),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Train the model using the tf.data.Dataset with augmentation
history_aug = model_aug.fit(train_dataset_augmented, # Use the augmented dataset
                            epochs=7
                            , # More epochs are often needed with data augmentation
                            validation_data=test_dataset_preprocessed) # Use the non-augmented test set

print("\n--- Scenario 3: Training with fine-tuning and data augmentation complete ---")
print(f"Scenario 3 - Final Validation Accuracy: {history_aug.history['val_accuracy'][-1]:.4f}")

Num GPUs Available:  1
TensorFlow is built with CUDA and should use GPU if available.

--- Data preparation complete. Images will be resized to (96, 96). ---
Train Dataset (augmented) batches: 1563
Test Dataset batches: 313

--- Starting Scenario 3: Fine-tuning with Data Augmentation (tf.data.Dataset) ---
Epoch 1/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 128s 76ms/step - accuracy: 0.5057 - loss: 1.4261 - val_accuracy: 0.7525 - val_loss: 0.7248
Epoch 2/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 128s 70ms/step - accuracy: 0.7747 - loss: 0.6626 - val_accuracy: 0.8017 - val_loss: 0.5900
Epoch 3/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 120s 76ms/step - accuracy: 0.8065 - loss: 0.5580 - val_accuracy: 0.8073 - val_loss: 0.5551
Epoch 4/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 141s 75ms/step - accuracy: 0.8283 - loss: 0.4867 - val_accuracy: 0.8210 - val_loss: 0.5119
Epoch 5/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 142s 75ms/step - accuracy: 0.8481 - loss: 0.4400 - val_accuracy: 0.8295 - val_loss: 0.4915
Epoch 6/7
1563/1563 ━━━━━━━━━━━━━